### Introduction
The sequences referred to in this assignment are all assumed to be DNA sequences with four bases, {A,C,T,G}.

In [1]:
import math
import random
import statistics

BASES = ['A', 'C', 'G', 'T']
GAP = '-'

def randomInt(_i):
    return int(math.floor(random.random() * _i))

# Random Choice from bases
def randomChoice(_bases):
    return _bases[randomInt(len(_bases))]

def randomSequence(_L, _bases):
    _list = []
    while len(_list) < _L:
        _list.append(randomChoice(_bases))
    return ''.join(_list)

def randomExp(lamda):
    return - math.log(1 - random.random()) / lamda

def poissonProcess(lamda, _T):
    eventTime = []
    t = randomExp(lamda)
    while t < _T: # draw exp(lamdaa) variables until we are past T
        eventTime.append(t)
        t += randomExp(lamda)
    return eventTime

def lenOfSequence(S):
    return int(sum(1 for s in S if s != GAP))

### Part 1
The Jukes-Cantor model of DNA sequence evolution is simple: each site mutates at rate $ \mu $ and when a mutation occurs, a new base is chosen uniformly at random from the four possible bases, ｛A; C; G; T｝. If we ignore mutations from base X to base X, the mutation rate is $\frac{3}{4}\mu$. All sites mutate independently of each other.

(a)

In [2]:
def evolveSequence(_A, mu, t, _bases):
    _A = list(_A)
    L = len(_A)
    lamda = L * 3/4 * mu
    for i in range(len(poissonProcess(lamda,t))):
        i = randomInt(L)
        # from the 3 possible bases to mutate
        tmp = list(_bases)
        tmp.remove(_A[i])
        _A[i] = randomChoice(tmp)

    return''.join(_A)

def simulateSiblings(L, mu, t, _bases):
    A = randomSequence(L, _bases)
    B = evolveSequence(A, mu, t, _bases)
    C = evolveSequence(A, mu, t, _bases)
    return A, B, C

L = 50
mu = 0.01
t = 10
A, B, C = simulateSiblings(L, mu, t, BASES)

In [3]:
print('A: ' + A)
print('B: ' + B)
print('C: ' + C)

A: CTATTGAGTTGATCTACGGAGGGGGCGAACATACGGGCATAAGGGTCTAG
B: CTATTGACTTGATCTACGGCGGGGGCGAACATACGGGCATAAGGGTCTAG
C: CTATTGAGTTGATATACGAAGGGGGCGCACATACTAGCATTAGGGTCTAG


(b)
The expected number of mutations for a single evolutionary process is $t\lambda = tL3/4\lambda$ and the expected number of mutations between two sibling sequences is $2tL34\mu$.

In [4]:
def countDiffs(A, B):
    return sum(1 for x, y in zip(A, B) if x != y)

print('Num of A diff B : ' + str(countDiffs(A, B)))
print('Num of A diff C : ' + str(countDiffs(A, C)))
print('Num of B diff C : ' + str(countDiffs(B, C)))

Num of A diff B : 2
Num of A diff C : 6
Num of B diff C : 8


In [5]:
n = 1000
L = 1000
mu = 0.01
t = 25
diffs = []
for _ in range(n):
    _, B, C = simulateSiblings(L, mu, t, BASES)
    diffs.append(countDiffs(B, C))

print('Mean: ' + str(statistics.mean(diffs)))
print('Variance: ' + str(statistics.variance(diffs)))

Mean: 294.387
Variance: 199.61084184184185


The mean or expected value of Poisson distributed is equal to its parameter, the sum of the independent Possion vars is also Possion distributed by the sum of parameters.  

The number of mutations is Poisson distributed with parameter, $2tL\frac{3}{4}\mu$ the number of differing due to the mutation could be happened at same site more than once.

(c)

In [6]:
L = 10000
mu = 0.03
t = 10
_, B, C = simulateSiblings(L, mu, t, BASES)
empiricalProbabilities = {}
theoreticalProbabilities = {}
for a in BASES:
    for b in BASES:
        empiricalProbabilities[(a,b)] = (sum(1 for x,y in zip(B,C) if (x,y) == (a,b)) / B.count(a) 
                                         + sum(1 for x,y in zip(B,C) if (x,y) == (b,a)) / B.count(b)) / 2
        theoreticalProbabilities[(a,b)] = 1/4 + (3/4 if a == b else -1/4) * math.exp(-2 * t * mu)

In [7]:
empiricalProbabilities

{('A', 'A'): 0.675751503006012,
 ('A', 'C'): 0.10958176515632892,
 ('A', 'G'): 0.10446101282613016,
 ('A', 'T'): 0.1137409784051133,
 ('C', 'A'): 0.10958176515632892,
 ('C', 'C'): 0.6621951219512195,
 ('C', 'G'): 0.11446974774407243,
 ('C', 'T'): 0.1129484538728576,
 ('G', 'A'): 0.10446101282613016,
 ('G', 'C'): 0.11446974774407243,
 ('G', 'G'): 0.6527279968140183,
 ('G', 'T'): 0.11856096788966747,
 ('T', 'A'): 0.1137409784051133,
 ('T', 'C'): 0.1129484538728576,
 ('T', 'G'): 0.11856096788966747,
 ('T', 'T'): 0.6617995264404104}

In [8]:
theoreticalProbabilities

{('A', 'A'): 0.6616087270705198,
 ('A', 'C'): 0.1127970909764934,
 ('A', 'G'): 0.1127970909764934,
 ('A', 'T'): 0.1127970909764934,
 ('C', 'A'): 0.1127970909764934,
 ('C', 'C'): 0.6616087270705198,
 ('C', 'G'): 0.1127970909764934,
 ('C', 'T'): 0.1127970909764934,
 ('G', 'A'): 0.1127970909764934,
 ('G', 'C'): 0.1127970909764934,
 ('G', 'G'): 0.6616087270705198,
 ('G', 'T'): 0.1127970909764934,
 ('T', 'A'): 0.1127970909764934,
 ('T', 'C'): 0.1127970909764934,
 ('T', 'G'): 0.1127970909764934,
 ('T', 'T'): 0.6616087270705198}

The result shows they are close.

===============================================================================================================
### Part 2
Insert or delete sequence.

Simulate insertions:
    draw a Poisson random variate, $hI$ , with rate $Lt\mu=10$ giving the number of insertions.

Simulate deletions: 
    draw a Poisson random variate, $hD$, with parameter $Lt\mu=10$ giving the number of deletions (use the $L$  for the ancestral sequence).
(a)

In [9]:
def simulateSiblingsInsertdelete(L, mu, t, _bases):
    A, B, C = map(list, simulateSiblings(L, mu, t, _bases))
    lamda = L * t * mu / 10
    h_I = len(poissonProcess(lamda,1))
    h_D = len(poissonProcess(lamda,1))
    print(h_I)
    print(h_D)
    for _ in range(h_D):
        # Choose i from non-gap sites
        i = min(randomInt(len(C)) + 1, len(C) - 3)
        C[i:i+3] = [GAP] * 3
    
    for _ in range(h_I):
        i = min(randomInt(len(B)) + 1, len(B) - 1)
        # Insertion
        B[i:i] = list(randomSequence(3, _bases))
        # Gaps for sibling
        C[i:i] = [GAP] * 3
    A, B, C = map(''.join, [A, B, C])
    return A, B, C

In [10]:
L = 50
mu = 0.01
t = 20
A, B, C = simulateSiblingsInsertdelete(L, mu, t, BASES)
print(A)
print(B)
print(C)

1
2
CCAGGATTAGATCGCTACATAGCCTTCGGAAGTATCGCGGGCTAATTTGT
CCAGTATGAGATCGCGAAATGGCCTTCGGAAGTGCCGCGGGCTCTAAAATGGT
CCAG---TAAATCGCAACATAGCCTTCGGAAGTATCGCGGGCT------TTGT


The output displays the result of insertion and deletion.

(b)    
The total number of events on the tree is no greater than Poisson distributed due to  the event rate is not constant through time.

===============================================================================================================
### Part 3

In [11]:
def alignOverlap(X, Y, S, d):
    # F[][] = 0
    F = [[0] * (len(X) + 1) for _ in range(len(Y) + 1)]

    for y in range(len(Y) + 1):
        for x in range(len(X) + 1):
            # F[0][] or F[][0] = 0
            if x == 0 or y == 0:
                F[y][x] = 0
            else:
                match = F[y-1][x-1] + S[Y[y-1]][X[x-1]]
                delete = F[y-1][x] + d
                insert = F[y][x-1] + d
                F[y][x] = max(match, delete, insert)

    # find i and j of Max of the F
    i = 0
    j = 0
    alignmentX = ''
    alignmentY = ''
    tmp = list(F[-1]) + [r[-1] for r in F]
    # print(tmp.index(max(tmp)) % len(F[-1]))
    if tmp.index(max(tmp)) > len(F[-1]):
        i = tmp.index(max(tmp)) % len(F[-1])
        j = len(X)
    else:
        i = len(Y)
        j = tmp.index(max(tmp))
    score = F[i][j]
    # Append Gaps
    if i + 1 < len(Y):
        alignmentX = GAP * (len(X) - i)
        alignmentY = Y[i:]
    elif j + 1 < len(X):
        alignmentX = X[j:]
        alignmentY = GAP * (len(Y) - j)

    # Backtrack to form alignment for overlapping region
    while i > 0 or j > 0: # match
        if i > 0 and j > 0 and F[i][j] == F[i-1][j-1] + S[Y[i-1]][X[j-1]]:
            alignmentX = X[j-1] + alignmentX
            alignmentY = Y[i-1] + alignmentY
            i -= 1
            j -= 1
        elif i > 0 and F[i][j] == F[i-1][j] + d: # deletion
            alignmentY = Y[i-1] + alignmentY
            alignmentX = GAP + alignmentX
            i -= 1
        elif j > 0 and F[i][j] == F[i][j-1] + d: # insertion
            alignmentY = GAP + alignmentY
            alignmentX = X[j-1] + alignmentX
            j -= 1
        else:
            break
            
    # Append Gaps
    if j > 0:
        alignmentX = X[:j] + alignmentX
        alignmentY = GAP * j + alignmentY
    elif i > 0:
        alignmentX = GAP * i + alignmentX
        alignmentY = Y[:i] + alignmentY

    return alignmentX, alignmentY, score

def scoreMatrix(_bases, score):
    return {x: {y: score[0] if x == y else score[1] for y in BASES} for x in BASES}

In [12]:
S = scoreMatrix(BASES, [2,-2])

Bp = B.replace(GAP, '')[:35]
Cp = C.replace(GAP, '')[-35:]

print('B: ' + B)
print('C: ' + C)

print('B\': ' + Bp)
print('C\': ' + Cp)

for d in range(-4, 0):
    alignmentB, alignmentC, score = alignOverlap(Bp, Cp, S, d)
    print('d = {}, score = {}'.format(d, score))
    print(alignmentB)
    print(alignmentC)

B: CCAGTATGAGATCGCGAAATGGCCTTCGGAAGTGCCGCGGGCTCTAAAATGGT
C: CCAG---TAAATCGCAACATAGCCTTCGGAAGTATCGCGGGCT------TTGT
B': CCAGTATGAGATCGCGAAATGGCCTTCGGAAGTGC
C': CGCAACATAGCCTTCGGAAGTATCGCGGGCTTTGT
d = -4, score = 26
CCAGTATGAGATCGCGAAATGGCCTTCGGAAGTGC------------
------------CGCAACATAGCCTTCGGAAGTATCGCGGGCTTTGT
d = -3, score = 27
CCAGTATGAGATCGCGAAATGGCCTTCGGAAGT-GC-----------
------------CGCAACATAGCCTTCGGAAGTATCGCGGGCTTTGT
d = -2, score = 30
CCAGTATGAGATCGCGAA-ATGGCCTTCGGAAGT-GC-----------
------------CGC-AACATAGCCTTCGGAAGTATCGCGGGCTTTGT
d = -1, score = 35
CCAGTATGAGATCGCGAA-ATGGCCTTCGGAAG--T-GC---------
------------CGC-AACATAGCCTTCGGAAGTATCGCGGGCTTTGT


In [13]:
B = 'ATCAGTTTGGAGAGGCCAGCTAAATCGCTGATGGTCACTGCCTCGTGCTT'
C = 'AGGAATGTTCAGCTAAAACGCTGATGGCCACTGACTTGTGATT'
S = scoreMatrix(BASES, [2,-2])
d = -3
Bp = B.replace(GAP, '')[:35]
Cp = C.replace(GAP, '')[-35:]
alignmentB, alignmentC, score = alignOverlap(Bp, Cp, S, d)
print('B: ' + B)
print('C: ' + C)

print('B\': ' + Bp)
print('C\': ' + Cp)

print(alignmentB)
print(alignmentC)

B: ATCAGTTTGGAGAGGCCAGCTAAATCGCTGATGGTCACTGCCTCGTGCTT
C: AGGAATGTTCAGCTAAAACGCTGATGGCCACTGACTTGTGATT
B': ATCAGTTTGGAGAGGCCAGCTAAATCGCTGATGGT
C': TCAGCTAAAACGCTGATGGCCACTGACTTGTGATT
ATCAGTTTGGAGAGGCCAGCTAAATCGCTGATGGT---------------
---------------TCAGCTAAAACGCTGATGGCCACTGACTTGTGATT


Run the part 3 for different d[-4:-1]

In [14]:
for d in range(-4, 0):
    alignmentB, alignmentC, score = alignOverlap(Bp, Cp, S, d)
    print('d = {}, score = {}'.format(d, score))
    print(alignmentB)
    print(alignmentC)

d = -4, score = 28
ATCAGTTTGGAGAGGCCAGCTAAATCGCTGATGGT---------------
---------------TCAGCTAAAACGCTGATGGCCACTGACTTGTGATT
d = -3, score = 28
ATCAGTTTGGAGAGGCCAGCTAAATCGCTGATGGT---------------
---------------TCAGCTAAAACGCTGATGGCCACTGACTTGTGATT
d = -2, score = 28
ATCAGTTTGGAGAGGCCAGCTAAATCGCTGATGGT----------------
---------------TCAGCTAAAACGCTGATGG-CCACTGACTTGTGATT
d = -1, score = 30
ATCAGTTTGGAGAGGC-CAGCTAAATCGCTGATGGT----------------
----------------TCAGCTAAAACGCTGATGG-CCACTGACTTGTGATT


d = -1 seems to giving the best result because the algorithm is able to identify the overlapping region. it got the highest score.